In [1]:
from glob import glob
import os
import pickle as pkl
import numpy as np
from IPython.display import Markdown as md

# Results of optimizing then testing

In [2]:
os.chdir("/Users/djpassey/Data/RCAlgorithm/results-1/")
results=[]
for path in glob("**/*.pkl", recursive=True):
    with open(path,'rb') as file:
        d=pkl.load(file)
        d["filename"]=path
        results.append(d)

In [33]:
INIT_COND_ROWS = {"random": 0, "activ_f": 2, "relax": 4}

TABLE_TEMPLATE = """| Mapping  |     | VPT |     | Error |     |
| --- | --- | ---:     | --- | ---:      | --- |
|            |  **Initial Cond.**    | Standard | Augmented | Standard | Augmented | 
| **Random**  | Continued | {} | {} | {} | {} |
|             | Arbitrary | {} | {} | {} | {} |
| **Activ_f** | Continued | {} | {} | {} | {} |
|             | Arbitrary | {} | {} | {} | {} |
| **Relax**   | Continued | {} | {} | {} | {} |
|             | Arbitrary | {} | {} | {} | {} |
"""

def experiment_attrs(fname):
    fname = fname.split("/")[1]
    syst, pred, initcond, training = fname.split("-")[:-1]
    return syst, pred, initcond, training

def experiment_table_indexes(fname):
    """ Returns the location where the experiment results belong in the data table
        Gives the location of the continued vpt for the given initial condition function
    """ 
    syst, initcond, pred, training = experiment_attrs(fname)
    row = INIT_COND_ROWS[initcond]
    col = 0
    if training == "augmented":
        col += 1
    return row, col

def extract_vpt_err(resultdicts):
    expdata = [
        (
            r["filename"], 
            np.array(r["continue"]),
            np.array(r["random"]),
            np.array(r["cont_deriv_fit"]), 
            np.array(r["rand_deriv_fit"])
        )
        for r in resultdicts if "continue" in r.keys()
    ]
    return expdata
        
def experiment_array(resultdicts, system, pred_type_optimized="random"):
    expdata = extract_vpt_err(resultdicts)
    E = np.zeros((6, 4))
    E[:, :] = np.nan
    for ed in expdata:
        fname = ed[0]
        syst, initcond, pred, training = experiment_attrs(fname)
        if (syst == system) and (pred == pred_type_optimized):
            row, col = experiment_table_indexes(fname)
            E[row, col] = np.mean(ed[1])
            E[row, col + 2] = np.mean(ed[3][:, 1])
            E[row + 1, col] = np.mean(ed[2])
            E[row + 1, col + 2] = np.mean(ed[4][:, 1])
    return E
    
def make_table(results, system, pred_type_optimized="continue"):
    header = f"## {system.title()} System (Optimized for predicition type: {pred_type_optimized})\n\n"
    E = experiment_array(results, system, pred_type_optimized)
    table = TABLE_TEMPLATE.format(*E.ravel())
    return md(header + table)

In [34]:
make_table(results, "lorenz")

## Lorenz System (Optimized for predicition type: continue)

| Mapping  |     | VPT |     | Error |     |
| --- | --- | ---:     | --- | ---:      | --- |
|            |  **Initial Cond.**    | Standard | Augmented | Standard | Augmented | 
| **Random**  | Continued | 1.1608 | 2.589200000000001 | 39.432546002377485 | 8.484151863424877 |
|             | Arbitrary | 0.0 | 0.0 | 947.2965306845332 | 1643.3667224280268 |
| **Activ_f** | Continued | 1.3083999999999991 | 2.2632 | 0.5961716057853503 | 14.616909483814272 |
|             | Arbitrary | 0.0 | 0.0 | 26137.48643403642 | 1049554.3449385941 |
| **Relax**   | Continued | 0.9319999999999993 | 1.6719999999999997 | 28.417679362861627 | 5.342681475283076 |
|             | Arbitrary | 0.10320000000000079 | 0.5924000000000009 | 79.0943955809891 | 15.751412732106122 |


### VPT
Best for **Continued:** `random, augmented` 

Best for **Arbitrary:** `relax, augmented`

### Error
Best for **Continued:** `activ_f, standard` 

Best for **Arbitrary:** `relax, augmented`


In [21]:
make_table(results, "lorenz", pred_type_optimized="random")

## Lorenz System (Optimized for predicition type: random)

|     |     | VPT |     | Error |     |
| --- | --- | ---:     | --- | ---:      | --- |
|            |  Initial Cond.    | Standard | Augmented | Standard | Augmented | 
| **Random**  | Continued | 0.4015999999999994 | 2.1824 | 397.8674290174672 | 29.35288298123386 |
|             | Arbitrary | 0.0 | 0.0 | 585.8179957143802 | 188.14830446150793 |
| **Activ_f** | Continued | 0.6423999999999992 | 0.42639999999999956 | 215.06212620121718 | 438.9757942637787 |
|             | Arbitrary | 0.006800000000000068 | 0.48120000000000007 | 369.60839793971104 | 530.7637615986717 |
| **Relax**   | Continued | 1.2264000000000004 | 0.9524000000000004 | 16.57146248716316 | 43.1178424138252 |
|             | Arbitrary | 0.402000000000001 | 0.058000000000000114 | 37.11714829618819 | 76.59278489176366 |


### VPT
Best for **Continued:** `random, augmented` 

Best for **Arbitrary:** `activ_f, augmented`

### Error
Best for **Continued:** `relax, standard` 

Best for **Arbitrary:** `relax, standard`


In [22]:
make_table(results, "rossler")

## Rossler System (Optimized for predicition type: continue)

|     |     | VPT |     | Error |     |
| --- | --- | ---:     | --- | ---:      | --- |
|            |  Initial Cond.    | Standard | Augmented | Standard | Augmented | 
| **Random**  | Continued | 45.153600000000004 | 48.0864 | 0.2974271037795056 | 0.15941392306484617 |
|             | Arbitrary | 0.0 | 0.0 | 134.22025229826073 | 65.92823098576692 |
| **Activ_f** | Continued | 47.7456 | 46.51480000000001 | 0.15741670089745113 | 0.1299668706324003 |
|             | Arbitrary | 0.0 | 52.10679999999999 | 275.4145253618851 | 1.5650546856011973 |
| **Relax**   | Continued | 17.785600000000002 | nan | 169.79754585257314 | nan |
|             | Arbitrary | 7.5376 | nan | 111.53703424279274 | nan |


In [23]:
make_table(results, "rossler", pred_type_optimized="random")

## Rossler System (Optimized for predicition type: random)

|     |     | VPT |     | Error |     |
| --- | --- | ---:     | --- | ---:      | --- |
|            |  Initial Cond.    | Standard | Augmented | Standard | Augmented | 
| **Random**  | Continued | 14.841600000000003 | nan | 14.189292135804044 | nan |
|             | Arbitrary | 0.32280000000000003 | nan | 64.71706739385249 | nan |
| **Activ_f** | Continued | 13.222800000000001 | nan | 8.356861828664217 | nan |
|             | Arbitrary | 1.5888000000000002 | nan | 33.59154730616775 | nan |
| **Relax**   | Continued | 21.954 | nan | 4.93168374235513 | nan |
|             | Arbitrary | 8.401200000000001 | nan | 7.888551921348621 | nan |


In [24]:
make_table(results, "thomas")

## Thomas System (Optimized for predicition type: continue)

|     |     | VPT |     | Error |     |
| --- | --- | ---:     | --- | ---:      | --- |
|            |  Initial Cond.    | Standard | Augmented | Standard | Augmented | 
| **Random**  | Continued | 87.73200000000004 | nan | 0.031678361481837246 | nan |
|             | Arbitrary | 0.0 | nan | 3.125434512770212 | nan |
| **Activ_f** | Continued | 110.49600000000004 | nan | 0.005465755653143769 | nan |
|             | Arbitrary | 2.600800000000002 | nan | 0.5223643136376134 | nan |
| **Relax**   | Continued | 72.12800000000006 | nan | 0.27880748383959303 | nan |
|             | Arbitrary | 45.711600000000004 | nan | 1.3630073250189307 | nan |


In [25]:
make_table(results, "thomas", pred_type_optimized="random")

## Thomas System (Optimized for predicition type: random)

|     |     | VPT |     | Error |     |
| --- | --- | ---:     | --- | ---:      | --- |
|            |  Initial Cond.    | Standard | Augmented | Standard | Augmented | 
| **Random**  | Continued | 39.28000000000005 | nan | 1.0593688373569983 | nan |
|             | Arbitrary | 0.06 | nan | 6.786469365742945 | nan |
| **Activ_f** | Continued | 45.30400000000005 | nan | 0.12700518915298795 | nan |
|             | Arbitrary | 7.272800000000002 | nan | 1.7503018094669067 | nan |
| **Relax**   | Continued | 49.19200000000004 | nan | 0.13852592431964778 | nan |
|             | Arbitrary | 40.7124 | nan | 0.3920901438088219 | nan |


In [28]:
make_table(results, "softrobot")

## Softrobo System (Optimized for predicition type: continue)

|     |     | VPT |     | Error |     |
| --- | --- | ---:     | --- | ---:      | --- |
|            |  Initial Cond.    | Standard | Augmented | Standard | Augmented | 
| **Random**  | Continued | nan | nan | nan | nan |
|             | Arbitrary | nan | nan | nan | nan |
| **Activ_f** | Continued | nan | nan | nan | nan |
|             | Arbitrary | nan | nan | nan | nan |
| **Relax**   | Continued | nan | nan | nan | nan |
|             | Arbitrary | nan | nan | nan | nan |


In [29]:
make_table(results, "softrobot", pred_type_optimized="random")

## Softrobo System (Optimized for predicition type: random)

|     |     | VPT |     | Error |     |
| --- | --- | ---:     | --- | ---:      | --- |
|            |  Initial Cond.    | Standard | Augmented | Standard | Augmented | 
| **Random**  | Continued | nan | nan | nan | nan |
|             | Arbitrary | nan | nan | nan | nan |
| **Activ_f** | Continued | nan | nan | nan | nan |
|             | Arbitrary | nan | nan | nan | nan |
| **Relax**   | Continued | nan | nan | nan | nan |
|             | Arbitrary | nan | nan | nan | nan |
